In [1]:
1+1

2

In [2]:
# %%
from utils import *
from trainer import Trainer
# %%
device = 'cuda:0'

base_model = HookedTransformer.from_pretrained(
    "pythia-160m-deduped", 
    device=device, 
)

checkpoint_mid_model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-160m-deduped", 
    device=device, 
    checkpoint_value = 512
)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model pythia-160m-deduped into HookedTransformer


Loaded pretrained model EleutherAI/pythia-160m-deduped into HookedTransformer


In [3]:
from datasets import load_dataset

In [13]:
pile_dedup_sample = load_dataset('EleutherAI/the_pile_deduplicated', streaming=True, split = 'train')

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

In [14]:
a = next(iter(pile_dedup_sample))

In [20]:
a_ten = checkpoint_mid_model.tokenizer(a["text"], return_tensors="pt")

In [22]:
a_ten["input_ids"].shape
# %%


torch.Size([1, 3147])

In [27]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm

def compile_all_tokens(sequence_length=256, batch_size=512, max_batches=1000, device='cuda:0'):
    """
    Iterates through the pile_dedup_sample dataset and compiles a new all_tokens tensor.

    Args:
        sequence_length (int): The length of each token sequence.
        batch_size (int): The number of sequences per batch.
        max_batches (int, optional): The maximum number of batches to process. Defaults to None.
        device (str): The device to store the tensor on.

    Returns:
        torch.Tensor: A tensor containing all tokenized sequences.
    """
    # Load dataset
    pile_dedup_sample = load_dataset('EleutherAI/the_pile_deduplicated', streaming=True, split='train')

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m-deduped')

    all_token_batches = []

    current_batch = []
    batch_count = 0

    print("Compiling all_tokens tensor...")
    for sample in tqdm(pile_dedup_sample, total=1650000):
        # Tokenize the text
        tokens = tokenizer.encode(sample['text'])
        
        # Get as many sequence_length chunks as possible from tokens
        for i in range(0, len(tokens) - sequence_length + 1, sequence_length):
            current_batch.append(tokens[i:i + sequence_length])
            
            # If batch is full, add to all_token_batches
            if len(current_batch) == batch_size:
                batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
                all_token_batches.append(batch_tensor)
                current_batch = []
                batch_count += 1
                # Check if we've reached the maximum number of batches
                if max_batches and batch_count >= max_batches:
                    break
        
        # Break outer loop if we've hit max_batches
        if max_batches and batch_count >= max_batches:
            break

    # Handle the last batch if it's not empty
    if current_batch:
        batch_tensor = torch.tensor(current_batch, dtype=torch.int32)
        all_token_batches.append(batch_tensor)

    # Concatenate all batches into a single tensor
    all_tokens = torch.cat(all_token_batches, dim=0).to(device)

    print(f"Compiled all_tokens tensor with shape: {all_tokens.shape}")
    return all_tokens

In [28]:
compile_all_tokens()

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

Compiling all_tokens tensor...


  0%|          | 31/1650000 [00:22<333:55:23,  1.37it/s]

torch.Size([512, 256])
tensor([[ 1147,   310,  2218,  ...,    15,  2064,  1453],
        [ 4566,  1411,   752,  ...,   690,  2238,   273],
        [47190,  2820,   281,  ..., 16323,   625,   390],
        ...,
        [ 1355, 18767,    13,  ...,    28, 50276,    73],
        [   24,  2617,    14,  ...,  6039,  3354, 18767],
        [14516,  1355,    14,  ...,    13,  5561,   285]], dtype=torch.int32)


0

In [7]:
pile_dedup_sample = load_dataset('EleutherAI/the_pile_deduplicated', streaming=True)

Resolving data files:   0%|          | 0/1650 [00:00<?, ?it/s]

(…)-00003-of-01650-8b668ee0e6c178f5.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

(…)-00004-of-01650-fc5086352c963b1a.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

(…)-00005-of-01650-c4cfa4a6dd83e624.parquet:   0%|          | 0.00/263M [00:00<?, ?B/s]

(…)-00006-of-01650-c0c63682a2c4f751.parquet:   0%|          | 0.00/274M [00:00<?, ?B/s]

(…)-00007-of-01650-bccb53b713a7cdbb.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
?load_dataset

Signature:
load_dataset(
    path: str,
    name: Optional[str] = None,
    data_dir: Optional[str] = None,
    data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None,
    split: Union[str, datasets.splits.Split, NoneType] = None,
    cache_dir: Optional[str] = None,
    features: Optional[datasets.features.features.Features] = None,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None,
    verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None,
    keep_in_memory: Optional[bool] = None,
    save_infos: bool = False,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    token: Union[bool, str, NoneType] = None,
    streaming: bool = False,
    num_proc: Optional[int] = None,
    storage_options: Optional[Dict] = None,
    trust_remote_code: bool = None,
   